
# Print the the graph in Neo4J
MATCH (n)
OPTIONAL MATCH (n)-[r]-(m)
RETURN n, r, m

# Delete the the graph in Neo4J
MATCH (n)
DETACH DELETE n


In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain_ollama langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain_community.vectorstores import Neo4jVector
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

from langchain_ollama import ChatOllama

from langchain.schema import Document
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer


from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

import re
import getpass
import os
from dotenv import load_dotenv


In [16]:
# Load environment variables from .env file
load_dotenv()

# Set them in the OS environment
os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")

#Instantiate the Graph DB
graph = Neo4jGraph()

In [10]:

### Data Formatting ###

# This text is form the website: https://huggingface.co/blog/open-r1
blog= f""" Open-R1: a fully open reproduction of DeepSeek-R1
What is DeepSeek-R1?
If you’ve ever struggled with a tough math problem, you know how useful it is to think a little longer and work through it carefully. OpenAI’s o1 model showed that when LLMs are trained to do the same—by using more compute during inference—they get significantly better at solving reasoning tasks like mathematics, coding, and logic.
However, the recipe behind OpenAI’s reasoning models has been a well kept secret. That is, until last week, when DeepSeek released their DeepSeek-R1 model and promptly broke the internet (and the stock market!).
Besides performing as well or better than o1, the DeepSeek-R1 release was accompanied by a detailed tech report that outlined the key steps of their training recipe. This recipe involved several innovations, most notably the application of pure reinforcement learning to teach a base language model how to reason without any human supervision. As shown in the figure below, making a powerful reasoning model is now very simple if you have access to a capable base model and a high-quality data mixture.

However, the DeepSeek-R1 release leaves open several questions about:
•	Data collection: How were the reasoning-specific datasets curated?
•	Model training: No training code was released by DeepSeek, so it is unknown which hyperparameters work best and how they differ across different model families and scales.
•	Scaling laws: What are the compute and data trade-offs in training reasoning models?

These questions prompted us to launch the Open-R1 project, an initiative to systematically reconstruct DeepSeek-R1’s data and training pipeline, validate its claims, and push the boundaries of open reasoning models. By building Open-R1, we aim to provide transparency on how reinforcement learning can enhance reasoning, share reproducible insights with the open-source community, and create a foundation for future models to leverage these techniques.

In this blog post we take a look at key ingredients behind DeepSeek-R1, which parts we plan to replicate, and how to contribute to the Open-R1 project.

How did they do it?
DeepSeek-R1 is a reasoning model built on the foundation of DeepSeek-V3. Like any good reasoning model, it starts with a strong base model, and DeepSeek-V3 is exactly that. This 671B Mixture of Experts (MoE) model performs on par with heavyweights like Sonnet 3.5 and GPT-4o. What’s especially impressive is how cost-efficient it was to train—just $5.5M—thanks to architectural changes like Multi Token Prediction (MTP), Multi-Head Latent Attention (MLA) and a LOT (seriously, a lot) of hardware optimization.

DeepSeek also introduced two models: DeepSeek-R1-Zero and DeepSeek-R1, each with a distinct training approach. DeepSeek-R1-Zero skipped supervised fine-tuning altogether and relied entirely on reinforcement learning (RL), using Group Relative Policy Optimization (GRPO) to make the process more efficient. A simple reward system was used to guide the model, providing feedback based on the accuracy and structure of its answers. This approach helped the model develop useful reasoning skills, such as breaking problems into steps and verifying its own outputs. However, its responses often lacked clarity and were difficult to read.
That’s where DeepSeek-R1 comes in. It started with a "cold start" phase, fine-tuning on a small set of carefully crafted examples to improve clarity and readability. From there, it went through more RL and refinement steps, including rejecting low-quality outputs with both human preference based and verifiable reward, to create a model that not only reasons well but also produces polished and consistent answers.
This all sounds great, but what's actually missing? Let's have a look at the missing pieces of the puzzle.
Open-R1: the missing pieces
The release of DeepSeek-R1 is an amazing boon for the community, but they didn’t release everything—although the model weights are open, the datasets and code used to train the model are not.
The goal of Open-R1 is to build these last missing pieces so that the whole research and industry community can build similar or better models using these recipes and datasets. And by doing this in the open, everybody in the community can contribute!
As shown in the figure below, here’s our plan of attack
•	Step 1: Replicate the R1-Distill models by distilling a high-quality reasoning dataset from DeepSeek-R1.
•	Step 2: Replicate the pure RL pipeline that DeepSeek used to create R1-Zero. This will involve curating new, large-scale datasets for math, reasoning, and code.
•	Step 3: Show we can go from base model → SFT → RL via multi-stage training.

The synthetic datasets will allow everybody to fine-tune existing or new LLMs into reasoning models by simply fine-tuning on them. The training recipes involving RL will serve as a starting point for anybody to build similar models from scratch and will allow researchers to build even more advanced methods on top.
Note that we don’t want to stop at math datasets. There’s a lot of potential in exploring other areas, obvious one like code but also scientific fields such as medicine, where reasoning models could have significant impact.
This initiative isn’t just about replicating results—it’s about sharing insights with the community. By documenting what works, what doesn’t, and why, we hope to save others from wasting time and compute on unproductive paths.
"""

### Text Splitting ###


### Split text based sentences ###
text_chunks = [sentence.strip() + "." for sentence in blog.split(".") if sentence.strip()]
# Convert text chunks into Document objects
documents = [Document(page_content=chunk) for chunk in text_chunks]

### Split text into smaller chunks ###
# splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=5)
# text_chunks = splitter.split_text(blog)  # Returns a list of strings
# documents = [Document(page_content=chunk) for chunk in text_chunks]

# For Wikipedia content uncomment this code
# raw_documents = WikipediaLoader(query="DeepSeek").load() #It get the webpage of DeepSeek.
# text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
# documents = text_splitter.split_documents(raw_documents)


In [13]:
### Graph Construction ###
#model_name="qwen2:7b"
#model_name="qwen2.5:32b"
model_name="mistral-small:24b"

# It initialise a language model.
llm = ChatOllama(model=model_name, temperature=0, format="json")
# It interacts with graphs or graph-like data structures using a LLM.
llm_transformer = LLMGraphTransformer(llm=llm,ignore_tool_usage=False) # ignore_tool_usage=False: For models that support tools set it to false. Otherwise, set it to true to allow prompt-based
# It processes textual data and converts it into a graph representation (identify entities, relationships, or concepts).
# Using async with LLM extraction is recommended, as it allows for parallel processing of multiple documents.
graph_documents = llm_transformer.convert_to_graph_documents(documents) #Synchronous
#graph_documents = await llm_transformer.aconvert_to_graph_documents(documents)

# It adds graph documents to a graph database.
# baseEntityLabel=True : the graph may include a generic label (e.g., Entity) for all nodes in addition to their specific labels (e.g., person or dynasty).
# include_source=True : Determines whether to include the source information (metadata) from the original documents in the graph.
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [14]:
### Ploting the Graph ###

# It finds relationships between nodes where the relationship type is NOT "MENTIONS".
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"
#default_cypher = "MATCH (s)-[r]->(t) WHERE NOT type(r) = 'MENTIONS' RETURN s,r,t LIMIT 50"

# GraphWidget: A visualisation tool for rendering and exploring graphs in Jupyter Notebooks.
# GraphDatabase: The official Neo4j driver for querying and updating graph data in a Neo4j database.
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase


# It checks if the script is running in Google Colab and, if so, enable support for custom widgets.
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

# It runs a Cypher query on a Neo4j graph database to retrieve data (nodes and relationships).
# It visualises the graph data interactively using GraphWidget from the yfiles_jupyter_graphs library.
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget


showGraph()

GraphWidget(layout=Layout(height='500px', width='100%'))

GraphWidget(layout=Layout(height='500px', width='100%'))